<a href="https://colab.research.google.com/github/veranika-izrailyan/Machine_Learning/blob/main/spark_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark - Preprocessing

#### Description
In this Hands-on lab, you will master your knowledge on PySpark, a very popular Python library for big data analysis and modeling. Here, you will learn how to create a dataset using the PySpark library, and to manipulate it using standard filtering and slicing techniques.

Your data management skills will be challenged, and by the end of this lab, you should have a deep understanding of how PySpark practically works to build data analysis pipelines.



#### Learning Objectives
Upon completion of this lab you will be able to:

- create a Spark Session, and store the data into a Spark DataFrame;
- query data with PySpark using standard SQL;
- create a new column inside the Spark DataFrame;
- perform standard data cleaning - type consistency, filtering, slicing;
- pivoting and manipulating a Spark DataFrame.

#### Intended Audience
This lab is intended for:
 - Those interested in performing data analysis with Python.
 - Anyone involved in data science and engineering pipelines.

#### Prerequisites
You should possess:
 - An intermediate understanding of Python.
 - Basic knowledge of SQL.
 - Basic knowledge of the following libraries: Pandas.



In [ ]:
# !pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u312-b07-1~deb9u1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

### Initialize a SparkSession

Let's introduce a very important object in `pyspark`: the `SparkSession`. The Spark Session is a unified entry point of a spark application from Spark 2.0. Before its introduction, the `SparkContext` was the entry point of any spark application, and needed a `SparkConf`, which had all the cluster configs and parameters to create a Spark Context object. 

We therefore import from the Spark SQL submodule the `SparkSession` class, and we instantiate a session using the `getOrCreate` method.

In [ ]:
# from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# conf = SparkConf().set("spark.ui.port", "4050")
# sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

Spark SQL provides `spark.read().csv("file_name")` to read a file or directory of files in CSV format into Spark DataFrame: let's use this method to ingest our dataset. We specify `header=True` to import the first row inside the csv as the header of the spark dataframe.

In [ ]:
file_path = "data/tips.csv"
tips = spark.read.csv(file_path, header=True)
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77|   2|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26|   5|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43|   3|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [ ]:
type(tips)

pyspark.sql.dataframe.DataFrame

As we can see, a simple inspection of the `tips` object confirms we are dealing with a Spark DataFrame.

Sound's good. But we are missing a crucial aspect, which can be explained as follows. Technically speaking, a Spark DataFrame is conceptually equivalent to a table in a relational database, and it is frequent to use this definition to identify a Spark DataFrame. In particular, Spark tables can be of two types. Temporary or Permanent. Both of these tables are present in a database. If we don’t specify any database, Spark uses the `default` database. We can see the list of available databases with `listDatabases`




In [ ]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/home/project/spark-warehouse')]

If we inspect the available Tables, we easily see that there are not tables available.

In [ ]:
spark.catalog.listTables('default')

[]

To create a table in the default database, we use the `createOrReplaceTempView` method

In [ ]:
tips.createOrReplaceTempView("tips") # Add tips data to the catalog

In [ ]:
spark.catalog.listTables()

[Table(name='tips', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

### Querying data with PySpark

The nice thing of Spark table is that we can run any query with standard SQL. So, for instance, suppose we wish to inspect the first 10 rows of the `tips` table. In standard SQL, this translates into the following query:

In [ ]:
QUERY_TIPS = "FROM tips SELECT * LIMIT 10"

we can now pass the query into the `sql` module, as follows:

In [ ]:
tips10 = spark.sql(QUERY_TIPS)
tips10.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77|   2|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+



There is a nice interaction between a Spark DataFrame (or table) and a Pandas DataFrame: indeed we can convert a Spark table into a Pandas DataFrame using the `toPandas` funciton. This can be useful, especially for data scientist who wish to use pandas in case they need  to perform further advanced manipualtions or simply to ingest in a more complex Python pipeline.

In [ ]:
tips10 = spark.sql(QUERY_TIPS)
tips10_df = tips10.toPandas()
tips10_df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.5,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2


Let us try to write a group by sql query as follows. We want to count the number of clients by day and gender. To do so, you have to write a query that performs the following operations:
 - SELECT the columns `day`, `sex` and `COUNT(*)`;
 - FROM the table `tips`;
 - GROUP BY `day` and `sex` columns;
 - ORDER BY the `day` column.


In [ ]:
STUDENT_QUERY = "SELECT day, sex, COUNT(*) as N FROM tips GROUP BY day, sex ORDER BY day"

In [ ]:
tips_counts = spark.sql(STUDENT_QUERY)
pd_counts = tips_counts.toPandas() # Convert the results to a pandas DataFrame
pd_counts.shape

(8, 3)

In [ ]:
# ====================================
# Validation Check
# DO NOT CHANGE THIS CELL
# ====================================
vcf_01 =  pd_counts.shape[0]
with open('results/vcf_01.txt', 'w') as f:
    f.write("%s\n" % vcf_01)

## Creating new columns with PySpark

Data wrangling with PySpark is pretty straigthforward. For instance, suppose we wish to add a new column to the spark table `tips`. To do so, we use the `withColumn` method, which returns a new DataFrame by adding a column or replacing the existing column that has the same name. The column expression must be an expression over this DataFrame.

So, for instance, suppose we wish to compute the tips' percentage over the total bill. we need two columns from the original table: the `tip` and the `total_bill`. So, inside the `withColumn`, we specify:
 - the name of the new columns as a string. In our case: `perc_tips`;
 - the column's expression based on existing tables' columns. In our case: `(tips.tip/tips.total_bill)*100`
 

In [ ]:
tips = spark.table("tips")
tips = tips.withColumn("perc_tips", (tips.tip/tips.total_bill)*100)
tips.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|         perc_tips|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77|   2|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26|   5|Female|    No|Sun|Dinner|   4|14.180374361883

### Pivoting a Spark Table

We can even perform data transformation directly with pyspark: for example, suppose we are interested in getting the total number of clients by their gender. We can call directly in the `tips` tabl;e the `groupBy` method, by specifying the column we wish to aggregate on - in our case `sex`.

To aggregate on the column `sex`, we use the `count()` method, which computes the total number of occurrnces by gender.

In [ ]:
by_sex = tips.groupBy("sex")
by_sex.count().show()

+------+-----+
|   sex|count|
+------+-----+
|Female|   87|
|  Male|  157|
+------+-----+



We can obviously specify a different aggregation method: for example, we might be intrested in computing the average percentage tips by gender.

In [ ]:
by_gender = tips.groupBy("sex")

In [ ]:
by_gender.avg("perc_tips").show()

+------+------------------+
|   sex|    avg(perc_tips)|
+------+------------------+
|Female|16.649073632892485|
|  Male|15.765054700429744|
+------+------------------+



### Converting types

Sometimes we want to be sure the data is stored correctly in our table. A good feature of the `withColumn` method is that it allows also to replace existing columns, whichm eans that we can even replace the existing type of the column. But how can we do that? 

Well, that's pretty easy: when calling the `withColumn` we specify the colum name and then the column's expression by applying the `cast` method on it. In this way, we can easily assign a new type to an existing column.

In [ ]:
tips = tips.withColumn("total_bill",tips.total_bill.cast("double"))
tips = tips.withColumn("tip", tips.tip.cast("double"))
tips = tips.withColumn("size", tips.size.cast("integer"))

In [ ]:
tips.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|         perc_tips|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

Let us try to perform an aggregation using the `groupBy` method as follows. We want to compute the average tip by gender and smoker. To do so, you have to apply the `groupBy` method on the `tips` by specifying, inside the `groupBy` call, the `"smoker"` and `"sex"` columns, separated by a comma. Be sure you store this object into the variable `by_smoker_sex_table`.

Then, apply on the `by_smoker_sex_table` the `avg("tip")`, which aggregates the tip value by smoker and gender. here the aggregation is the average tip with respect to the specified columns. Be sure you call the `show()` method at the end.

In [ ]:
by_smoker_sex_table = tips.groupBy("smoker", "sex")
by_smoker_sex_table.avg("tip").show()

+------+------+------------------+
|smoker|   sex|          avg(tip)|
+------+------+------------------+
|    No|Female|2.7735185185185185|
|    No|  Male|  3.11340206185567|
|   Yes|  Male|3.0511666666666666|
|   Yes|Female| 2.931515151515151|
+------+------+------------------+



In [ ]:
by_smoker_sex_df = by_smoker_sex_table.avg("tip").toPandas() # Convert the results to a pandas DataFrame
by_smoker_sex_df.iloc[0,1] # must be "Female"

'Female'

### Slicing a Tabel with `select`

In case we wish to select just a few columns, we can use the `select` method, which allows to select the desired columns, and it returns a temporary view.

In [ ]:
# Select the correct columns
tips_000 = tips.select("total_bill", "tip", "size", "perc_tips")
tips_000.show()

+----------+----+----+------------------+
|total_bill| tip|size|         perc_tips|
+----------+----+----+------------------+
|     16.99|1.01|   2|5.9446733372572105|
|     10.34|1.66|   3|16.054158607350097|
|     21.01| 3.5|   3|16.658733936220845|
|     23.68|3.31|   2| 13.97804054054054|
|     24.59|3.61|   4|14.680764538430255|
|     25.29|4.71|   4| 18.62396204033215|
|      8.77| 2.0|   2| 22.80501710376283|
|     26.88|3.12|   4|11.607142857142858|
|     15.04|1.96|   2|13.031914893617023|
|     14.78|3.23|   2|21.853856562922868|
|     10.27|1.71|   2| 16.65043816942551|
|     35.26| 5.0|   4|14.180374361883155|
|     15.42|1.57|   2|10.181582360570687|
|     18.43| 3.0|   4|16.277807921866522|
|     14.83|3.02|   2|20.364126770060686|
|     21.58|3.92|   2|18.164967562557923|
|     10.33|1.67|   3| 16.16650532429816|
|     16.29|3.71|   3|22.774708410067525|
|     16.97| 3.5|   3|20.624631703005306|
|     20.65|3.35|   3|16.222760290556902|
+----------+----+----+------------

### Filtering a Table with `filter`

We have two distinct ways to perform filtering with a Spark table:
 1. by using a string expression inside the `filter` method;
 2. by using a boolean condition inside the `filter` method.

In particular, here we want to perform a simple filtering condition, namely picking all records with a `total_bill` greater than 40 USD.

#### Filtering with a string condition

In [ ]:
tips_01 = tips.filter("total_bill>40").toPandas()
tips_01

,total_bill,tip,sex,smoker,day,time,size,perc_tips
0,48.27,6.73,Male,No,Sat,Dinner,4,13.942407
1,40.17,4.73,Male,Yes,Fri,Dinner,4,11.774956
2,44.30,2.50,Female,Yes,Sat,Dinner,3,5.643341
3,41.19,5.00,Male,No,Thur,Lunch,5,12.138869
4,48.17,5.00,Male,No,Sun,Dinner,6,10.379905
5,50.81,10.00,Male,Yes,Sat,Dinner,3,19.681165
6,45.35,3.50,Male,Yes,Sun,Dinner,3,7.717751
7,40.55,3.00,Male,Yes,Sun,Dinner,2,7.398274
8,43.11,5.00,Female,Yes,Thur,Lunch,4,11.598237
9,48.33,9.00,Male,No,Sat,Dinner,4,18.621974


#### Filtering with a Boolean condition

In [ ]:
tips_02 = tips.filter(tips.total_bill>40).toPandas() # Filter tips with a boolean
tips_02

,total_bill,tip,sex,smoker,day,time,size,perc_tips
0,48.27,6.73,Male,No,Sat,Dinner,4,13.942407
1,40.17,4.73,Male,Yes,Fri,Dinner,4,11.774956
2,44.30,2.50,Female,Yes,Sat,Dinner,3,5.643341
3,41.19,5.00,Male,No,Thur,Lunch,5,12.138869
4,48.17,5.00,Male,No,Sun,Dinner,6,10.379905
5,50.81,10.00,Male,Yes,Sat,Dinner,3,19.681165
6,45.35,3.50,Male,Yes,Sun,Dinner,3,7.717751
7,40.55,3.00,Male,Yes,Sun,Dinner,2,7.398274
8,43.11,5.00,Female,Yes,Thur,Lunch,4,11.598237
9,48.33,9.00,Male,No,Sat,Dinner,4,18.621974


We can even apply multiple filtering as follows:

In [ ]:
filterA = tips.sex == "Female"
filterB = tips.day == "Sun"
tips_03 = tips.filter(filterA).filter(filterB)

In [ ]:
tips_03.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|         perc_tips|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883155|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|20.364126770060686|
|     10.33|1.67|Female|    No|Sun|Dinner|   3| 16.16650532429816|
|     16.97| 3.5|Female|    No|Sun|Dinner|   3|20.624631703005306|
|     10.29| 2.6|Female|    No|Sun|Dinner|   2| 25.26724975704568|
|     34.81| 5.2|Female|    No|Sun|Dinner|   4|14.938236139040505|
|     25.71| 4.0|Female|    No|Sun|Dinner|   3|15.558148580318942|
|     17.31| 3.5|Female|    No|Sun|Dinner|   2|20.219526285384173|
|     29.85|5.14|Female|    No|Sun|Dinner|   5|17.219430485762143|
|      25.0|3.75|Female|    No|Sun|Dinner|   4|              1

Another nice application of this feature is that we can remove null values with a simple string. For example, in case we wish to remove all reocrds where the `total_bill` is null, we just need to pass to the `filter` call the string `"total_bill is not NULL"`. That's cool, isnt't it?

In [ ]:
tips_counts = tips.filter("total_bill is not NULL")
tips_counts.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|         perc_tips|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

Let us try to perform an aggregation using the `groupBy` method as follows. Similarly to what we did before, we want to compute the average tip for a non-smoker Male. To do so, we can filter the `tips` by gender and smoker, as follows:
 - filter by `tips.sex=='Male'`
 - filter by `tips.smoker=='No'`

and then you `groupBy().avg('perc_tips')`. Be sure you call the `show()` method at the end.

In [ ]:
exercise_table = tips.filter(tips.sex=='Male').filter(tips.smoker=='No').groupBy().avg('perc_tips')
exercise_table_df = exercise_table.toPandas()
exercise_table_df

,avg(perc_tips)
0,16.066872


In [ ]:
# ====================================
# Validation Check
# DO NOT CHANGE THIS CELL
# ====================================
vcf_02 =  int(exercise_table_df.iloc[0,0])
with open('results/vcf_02.txt', 'w') as f:
    f.write("%s\n" % vcf_02)

**End Lab**